In [ ]:
import datacube
from datacube.utils import cog
from datetime import datetime
from datetime import timedelta
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from datacube.utils.masking import make_mask
from datacube.utils.masking import mask_invalid_data
from odc.algo import mask_cleanup
import sys
sys.path.insert(1, '../Tools/')
from dea_tools.datahandling import load_ard
from dea_tools.plotting import rgb, display_map
from dea_tools.bandindices import calculate_indices
from dea_tools.dask import create_local_dask_cluster
from dea_tools.spatial import xr_vectorize
import scipy.ndimage
# Create local dask cluster to improve data load time
client = create_local_dask_cluster(return_client=True)

In [ ]:
dc = datacube.Datacube(app="bais2_mapping")

In [ ]:
# Define the area of interest
latitude = (-28.1218076, -28.2168763)
longitude = (153.1025233, 153.2382923)

# Set the range of dates for the complete sample
time = ('2019-08-20', '2019-12-01')
# Compute the bounding box for the study area
study_area_lat = (-28.1218076, -28.2168763)
study_area_lon = (153.1025233, 153.2382923)

display_map(x=study_area_lon, y=study_area_lat, margin=-0.2)

In [ ]:
# Fire event date
fire_date = '2019-09-06'

# Length of baseline period
fire_length = '3 months'

In [ ]:
# Define dates for loading data
if fire_length == '12 months':
    time_step = timedelta(days=365)
if fire_length == '6 months':
    time_step = timedelta(days=182.5)
if fire_length == '3 months':
    time_step = timedelta(days=91)

# Calculate the start and end date for baseline data load
start_date_pre = datetime.strftime(
    ((datetime.strptime(fire_date, '%Y-%m-%d')) - time_step), '%Y-%m-%d')
end_date_pre = datetime.strftime(
    ((datetime.strptime(fire_date, '%Y-%m-%d')) - timedelta(days=1)),
    '%Y-%m-%d')

# Calculate end date for post fire data load
start_date_post = datetime.strftime(
    ((datetime.strptime(fire_date, '%Y-%m-%d')) + timedelta(days=15)),
    '%Y-%m-%d')
end_date_post = datetime.strftime(
    ((datetime.strptime(fire_date, '%Y-%m-%d')) + timedelta(days=90)),
    '%Y-%m-%d')


# Print dates
print(f'start_date_pre:  {start_date_pre}')
print(f'end_date_pre:    {end_date_pre}')
print(f'fire_date:       {fire_date}')
print(f'start_date_post: {start_date_post}')
print(f'end_date_post:   {end_date_post}')

In [ ]:
# Band List
#dc_measurements = dc.list_measurements()
#dc_measurements.loc[['ga_s2am_ard_3']]

In [ ]:
resolution = (-20, 20)
measurements = ['nbart_red_edge_2', 'nbart_red_edge_3', 'nbart_nir_2', 'nbart_red', 'nbart_swir_3', 'nbart_nir_1', 'nbart_green', 'nbart_blue','fmask' ]
min_gooddata = 0.8
output_crs = 'EPSG:9473'

In [ ]:
# Load all data in baseline period available from ARD data
dsbaseline = load_ard(dc=dc,
                        products=['ga_s2am_ard_3', 'ga_s2bm_ard_3'],
                        x=study_area_lon,
                        y=study_area_lat,
                        time=(start_date_pre, end_date_pre),
                        measurements=measurements,
                        min_gooddata=min_gooddata,
                        resampling={
                         "fmask": "nearest",
                          "*": "bilinear"},
                        output_crs=output_crs,
                        resolution=resolution,
                        group_by='solar_day')


In [ ]:
#selecting best timestep
rgb(dsbaseline,col="time")

In [ ]:
dsbaseline.fmask.attrs["flags_definition"]

In [ ]:
#dsbaseline.fmask.plot(col="time", col_wrap=4)

In [ ]:
# Create the mask based on "valid" pixels
clear_mask = make_mask(dsbaseline.fmask, fmask="valid")
#clear_mask.plot(col="time", col_wrap=4)

In [ ]:
# Apply the mask
baseclear = dsbaseline.where(clear_mask)
#rgb(baseclear, col="time")

In [ ]:
%matplotlib inline

import datacube
import geopandas as gpd
import odc.geo.xr
from odc.geo.geom import Geometry

import sys

sys.path.insert(1, "../Tools/")
from dea_tools.plotting import rgb

In [ ]:
polygon_to_drill = (
    "lamington_boundary.geojson"
)
# Read vector file
polygon_to_drill = gpd.read_file(polygon_to_drill)

# Check that the polygon loaded as expected. We'll just print the first 3 rows to check
#polygon_to_drill.head(3)
# Select polygon
shapely_geometry = polygon_to_drill.iloc[0].geometry

# Convert to Geometry object with CRS information
geom = Geometry(geom=shapely_geometry, crs=polygon_to_drill.crs)
geom

In [ ]:
# Mask out all pixels outside of our polygon:
base_fullymasked = baseclear.odc.mask(poly=geom)
#rgb(base_fullymasked, col="time")

In [ ]:
# Set the timesteps to visualise
best_timestep = 21

# Generate RGB plot of best timestamp
rgb(base_fullymasked, index=[best_timestep])

In [ ]:
#defining bais2 calculation
B06=base_fullymasked.nbart_red_edge_2/10000
B07=base_fullymasked.nbart_red_edge_3/10000
B8A=base_fullymasked.nbart_nir_2/10000
B04=base_fullymasked.nbart_red/10000
B12=base_fullymasked.nbart_swir_3/10000

#applying the calculation to the bands in ds then adding it as a variable in that ds
base_fullymasked['BAIS2'] = (1-((B06*B07*B8A)/B04)**0.5)*((B12-B8A)/((B12+B8A)**0.5)+1)

In [ ]:
#selecting the baseline timestep
baseline_BAI = base_fullymasked.BAIS2.isel(time=21)
# Select RGB baseline
baseline_image = base_fullymasked.isel(time=21)

In [ ]:
#plotting histogram of baseline_BAI, used to set vmin and vmax
plt.hist(baseline_BAI.data.flatten(),bins=np.arange(-1,1,0.1))

In [ ]:
#Plotting the baseline comparison
# Set up subplots for baseline
f, axarr = plt.subplots(1, 2, figsize=(15, 7), squeeze=False)

# Visualise post-fire image as a true colour image
rgb(baseline_image, 
    bands=['nbart_red', 'nbart_green', 'nbart_blue'], 
    ax=axarr[0, 0])
axarr[0, 0].set_title('Pre-fire RGB')
axarr[0, 0].set_xlabel('X coordinate')
axarr[0, 0].set_ylabel('Y coordinate')

# Visualise post-fire image as BAI image
baseline_BAI.plot(cmap='RdBu', vmin=-1, vmax=1, ax=axarr[0, 1])
axarr[0, 1].set_title('Pre-fire BAIS2')
axarr[0, 1].yaxis.set_visible(True)
axarr[0, 1].set_xlabel('X coordinate');



In [ ]:
#Preparing the post-fire images
#Load post-fire data from Sentinel-2A and 2B
# Load all data in baseline period available from ARD data
dspost = load_ard(dc=dc,
                        products=['ga_s2am_ard_3', 'ga_s2bm_ard_3'],
                        x=study_area_lon,
                        y=study_area_lat,
                        time=('2019-12-5', '2020-01-30'),
                        measurements=measurements,
                        min_gooddata=min_gooddata,
                        resampling={
                         "fmask": "nearest",
                          "*": "bilinear"},
                        output_crs=output_crs,
                        resolution=resolution,
                        group_by='solar_day')
# Load all data in baseline period available from ARD data


In [ ]:
#selecting post fire image
rgb(dspost,col="time")

In [ ]:
dspost.fmask.attrs["flags_definition"]

In [ ]:
clear_mask_post = make_mask(dspost.fmask, fmask="valid")
#clear_mask.plot(col="time", col_wrap=4)

In [ ]:
postclear = dspost.where(clear_mask_post)

In [ ]:
# Mask out all pixels outside of our polygon:
post_fullymasked = postclear.odc.mask(poly=geom)
#rgb(post_fullymasked, col="time")

In [ ]:
# Set the timesteps to visualise
best_timestep1 = 7

# Generate RGB plot of best timestamp
rgb(post_fullymasked, index=[best_timestep1])

In [ ]:
#defining bais2 calculation
B06=post_fullymasked.nbart_red_edge_2/10000
B07=post_fullymasked.nbart_red_edge_3/10000
B8A=post_fullymasked.nbart_nir_2/10000
B04=post_fullymasked.nbart_red/10000
B12=post_fullymasked.nbart_swir_3/10000

#applying the calculation to the bands in ds then adding it as a variable in that ds
post_fullymasked['BAIS2'] = (1-((B06*B07*B8A)/B04)**0.5)*((B12-B8A)/((B12+B8A)**0.5)+1)

In [ ]:
post_fullymasked

In [ ]:
#selecting the baseline timestep
post_BAI = post_fullymasked.BAIS2.isel(time=best_timestep1)
# Select RGB baseline
post_image = post_fullymasked.isel(time=best_timestep1)

In [ ]:
#post_BAI.data.flatten()
#plotting histogram of baseline_BAI, used to set vmin and vmax
plt.hist(post_BAI.data.flatten(),bins=np.arange(-1,1.4,0.1))

In [ ]:
# Set up subplots
f, axarr = plt.subplots(1, 2, figsize=(15, 7), squeeze=False)

# Visualise post-fire image as a true colour image
rgb(post_image, 
    bands=['nbart_red', 'nbart_green', 'nbart_blue'], 
    ax=axarr[0, 0])
axarr[0, 0].set_title('Post-fire RGB')
axarr[0, 0].set_xlabel('X coordinate')
axarr[0, 0].set_ylabel('Y coordinate')

# Visualise post-fire image as BAI image
post_BAI.plot(cmap='RdBu', vmin=0.5, vmax=0.8, ax=axarr[0, 1])
axarr[0, 1].set_title('Post-fire BAIS2')
axarr[0, 1].yaxis.set_visible(True)
axarr[0, 1].set_xlabel('X coordinate');


In [ ]:
#creating delta of baseline and post images
delta_BAI = post_BAI - baseline_BAI

# Visualise dBAIS2 image
delta_BAI.plot(cmap='RdBu_r', vmin=-0.3, vmax=0.6, figsize=(11, 9))
plt.xlabel('X coordinate')
plt.ylabel('Y coordinate');

In [ ]:
#plot histogram
plt.hist(delta_BAI.data.flatten(),bins=np.arange(-0.75,0.6,0.1))


In [ ]:
bam = delta_BAI
# Create a figure to plot the chosen fire severity index
f, axarr = plt.subplots(
    1, 2, figsize=(15, 10), squeeze=False, gridspec_kw={"width_ratios": [1, 5]}
)

# Calculate and round the dBAI dataarray value range to set determine the plots colourmap range
bam_BAI_min = round(float(bam.quantile(0.005)), 1)
bam_BAI_max = round(float(bam.quantile(0.995)), 1)

# PLot the dBAI dataarray on the second subplot of the above figure
bam.plot(cmap="RdBu_r", vmin=bam_BAI_min, vmax=bam_BAI_max, ax=axarr[(0, 1)])

# Plot a histogram of dBAI values in the first figure subplot.
# Calculate a colourmap from the dataarray plot by iterating through individual histogram patches
cm = plt.colormaps.get_cmap("RdBu_r")

n, bins, patches = xr.plot.hist(
    darray=bam,
    bins=np.arange(bam_BAI_min, bam_BAI_max + 0.05, 0.05),
    align="mid",
    orientation="horizontal",
    ec="black",
    yticks=(np.arange(bam_BAI_min - 0.05, bam_BAI_max + 0.05, step=0.05)),
    ax=axarr[(0, 0)],
)

# Match the colour scale of the histogram to that used in the map plot.
bin_centers = 0.5 * (bins[:-1] + bins[1:])
col = bin_centers - min(bin_centers)
col /= max(col)
for c, p in zip(col, patches):
    plt.setp(p, "facecolor", cm(c))

# Set titles for each subplot
axarr[0, 0].set_title('dBAIS2' + " Histogram")
axarr[0, 1].set_xlabel('X Coordinate')
axarr[0, 1].set_ylabel('Y Coordinate')
axarr[0, 1].set_title(
    'dBAIS2'
    + " measured between "
    + str(baseline_BAI.time.values)[:10]
    + " - "
    + str(post_BAI.time.values)[:10]
)

# Set the x-axis label and number of x-axis ticks for the histogram plot
axarr[0, 0].set_xlabel('dBAIS2' + " count")
axarr[0, 0].xaxis.set_major_locator(plt.MaxNLocator(3))

In [ ]:
#create pixel classes
unburnt = delta_BAI < 0.2
low = (delta_BAI > 0.2) & (delta_BAI < 0.3)
moderate = (delta_BAI > 0.3) & (delta_BAI < 0.4)
high = (delta_BAI > 0.4) & (delta_BAI < 0.45)
extreme = delta_BAI > 0.45

In [ ]:
import xarray as xr

# Assuming 'delta_BAI' is a NumPy array
#delta_BAI = xr.DataArray(delta_BAI, dims=['636', '739'])  # Replace 'dim1', 'dim2' with actual dimension names

In [ ]:
# Multiply by 1000 to match the classification ranges
BAI_scaled = delta_BAI * 1000

# Define the conditions and corresponding values using EU classes (https://forest-fire.emergency.copernicus.eu/about-effis/technical-background/fire-severity)
conditions = [
    (BAI_scaled < 200),
    (BAI_scaled >= 200) & (BAI_scaled < 300),
    (BAI_scaled >= 300) & (BAI_scaled < 400),
    (BAI_scaled >= 400) & (BAI_scaled < 450),
    (BAI_scaled >= 450)
]

values = [1, 2, 3, 4, 5]

# Create a new DataArray with the classified values
BAI_classified = xr.DataArray(
    np.select(conditions, values),
    coords=delta_BAI.coords,
    dims=delta_BAI.dims,
    name='BAI_Class'
)

# Add attributes for class descriptions
BAI_classified.attrs['classes'] = {
    1: 'Unburned or Regrowth',
    2: 'Low severity',
    3: 'Moderate low severity',
    4: 'Moderate high severity',
    5: 'High severity'
}

In [ ]:
pip install matplotlib-scalebar

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib_scalebar.scalebar import ScaleBar

# Assuming you have already created BAI_classified as in the previous example

# Define colors for each class
colors = ['olivedrab', 'yellow', 'orange', 'red', 'darkred']
cmap = ListedColormap(colors)

# Set up the colorbar ticks and boundaries
bounds = np.arange(0.5, 6.5, 1)  # 0.5, 1.5, 2.5, 3.5, 4.5, 5.5
norm = BoundaryNorm(bounds, cmap.N)

# Create the plot
fig, ax = plt.subplots(figsize=(12, 10))

# Plot the data
im = ax.imshow(BAI_classified, cmap=cmap, norm=norm)

# Add colorbar
cbar = fig.colorbar(im, ax=ax, orientation='vertical', pad=0.02)

# Set colorbar ticks and labels
cbar.set_ticks(np.arange(1, 6))
cbar.set_ticklabels(['Unburned or Regrowth', 'Low severity', 'Moderate low severity', 
                     'Moderate high severity', 'High severity'])

# Set title and labels
ax.set_title('Burn Severity Map (dBAIS2)', fontsize=16)

# Remove x and y ticks
ax.set_xticks([])
ax.set_yticks([])

# Add a scale bar
scalebar = ScaleBar(1, location='lower right')  # You can adjust the value to represent the map scale
ax.add_artist(scalebar)

# Add a north arrow
# Define the position and size of the north arrow
arrow_x = 0.9  # x position (in axis coordinates, from 0 to 1)
arrow_y = 0.9  # y position (in axis coordinates, from 0 to 1)
arrow_length = 0.1  # Length of the arrow in axis coordinates

x, y, arrow_length = 0.05, 0.95, 0.07
ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor='black', width=5, headwidth=15),
            ha='center', va='center', fontsize=20,
            xycoords=ax.transAxes)

plt.tight_layout()
plt.show()

In [ ]:
#masking the final burn scar

# Set threshold
threshold = 0.2


# Apply threshold
burnt = delta_BAI > threshold

delta_BAI['burnt'] = delta_BAI < threshold
#delta_BAIt = delta_BAI.burnt


total_burn = delta_BAI.where(burnt==1)
# Visualise dBAI image

total_burn.plot(cmap='inferno', figsize=(11, 9), vmax=0.6, vmin=-0.2)
plt.title('dBAIS2 Burn Area')
plt.xlabel('X coordinate')
plt.ylabel('Y coordinate');

In [ ]:
from scipy import ndimage
from skimage import morphology

sys.path.insert(1, "../Tools/")
from dea_tools.bandindices import calculate_indices
from dea_tools.datahandling import load_ard
from dea_tools.plotting import display_map, rgb, plot_variable_images
from dea_tools.spatial import xr_rasterize, xr_vectorize

In [ ]:
#salt/pepper clean?


In [ ]:
#Calculating Burnt Area

In [ ]:
# Constants for calculating burnt area
pixel_length = resolution[1]  # in metres
m_per_km = 1000  # conversion from metres to kilometres

# Area per pixel
area_per_pixel = pixel_length ** 2 / m_per_km ** 2

# Calculate areas
unburnt_area = (delta_BAI <= threshold).sum() * area_per_pixel
burnt_area = burnt.sum() * area_per_pixel
not_nan_area = delta_BAI.notnull().sum() * area_per_pixel
nan_area = delta_BAI.isnull().sum() * area_per_pixel
total_area = unburnt_area + burnt_area + nan_area

print(f'Unburnt area:            {unburnt_area.item():.2f} km^2')
print(f'Burnt area:              {burnt_area.item():.2f} km^2')
print(f'Nan area:                {nan_area.item():.2f} km^2')
print(f'Total area (no nans):    {not_nan_area.item():.2f} km^2')
print(f'Total area (with nans):  {total_area.item():.2f} km^2')

In [ ]:
#download as shapefile
from dea_tools.spatial import xr_rasterize, xr_vectorize
# Convert the burnt area from raster to vector format
gdf = xr_vectorize(
    da=total_burn, output_path= "total_burnnn.shp"
)

In [ ]:
#vectorise
#polygons = xr_vectorize(total_burn)
#polygons["area"] = polygons.area
#polygons["size"] = np.where(polygons["area"] < 50000, 1, 2)  # se

In [ ]:
#export burnt area map to test
from datacube.utils.cog import write_cog
# Write GeoTIFF
write_cog(geo_im=delta_BAI,
          fname='burnt_area_test.tif',
          overwrite=True,
          nodata=0.0)